In [1]:
import numpy as np
import matplotlib as plt 

import tensorflow as tf 

Tensorflow는 gradient 계산 시에 forward 과정에서 순차적으로 어떤 노드를 거쳤는지 기억해서 이를 역순으로 gradient descent를 거칩니다. 이때 사용하는 것이 gradient tape라고 합니다. 
tf.GradientTape는 context 안에서 실행되는 모든 연산을 tape에 record합니다. 그리고 기록된 연산의 그레디언트를 계산합니다. 

In [3]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2 

In [4]:
dy_dx = tape.gradient(y, x) #이때 GradientTape.gradient(target, sources)에서 target은 실제 y_hat , x는 입력값이라고 생각하면 됩니다.


In [11]:
w = tf.Variable(tf.random.normal((3,2)), name="w") 
b = tf.Variable(tf.zeros(2,dtype=tf.float32), name='b')
x= [[1. ,2. ,3.]]

with tf.GradientTape(persistent=True) as tape: # persistent = True로 진행한다면, 같은 computation을 여러 번 진행하기 위해서입니다.
    y = tf.matmul(x,w) + b
    loss = tf.reduce_mean(y**2) # 
grad = tape.gradient(loss, [w,b] )# 한 개보다 많은 변수에 대해 backward 과정을 거칩니다. 
# 이때 tape.gradient에 backward를 해야하는 변수들을 리스트 형태로 넣기에 
# grad[0] 에는 w, grad[1]에는 b가 저장된다. 
grad

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.03123844, 2.434194  ],
        [0.06247687, 4.868388  ],
        [0.09371531, 7.3025823 ]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.03123844, 2.434194  ], dtype=float32)>]

In [12]:
grad_options = {
    "w": w,
    "b": b
}
grad_dictionary = tape.gradient(loss, grad_options)
grad_dictionary['w'] #dictionary 형태로 tape.gradient에 넣으면 dictionary 형태로 접근할 수 있습니다.

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.03123844, 2.434194  ],
       [0.06247687, 4.868388  ],
       [0.09371531, 7.3025823 ]], dtype=float32)>

In [15]:
# model에서 사용할 gradient 
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2. , 3.]])

with tf.GradientTape() as tape:

    y = layer(x) 
    loss = tf.reduce_mean(y**2) #cost function which is sum of error 
grad = tape.gradient(loss, layer.trainable_variables) # train할 수 있는 layer의 변수들을 넣어주는 것이다.


In [17]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}') # Kernel은 layer에서 정해진 weight matrix
                                         # kernel is a weights matrix created by the layer,

dense_2/kernel:0, shape: (3, 2)
dense_2/bias:0, shape: (2,)


In [18]:
# gradient에 unconnect일때 None이 나오는 대신 0을 가져오는 것으로 바꿀 수 있습니다.
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))

tf.Tensor([0. 0.], shape=(2,), dtype=float32)


In [20]:
# Gradient Tape 에서 gradient descent 할 변수와 하지 않을 변수를 따로 결정할 수 있습니다. 
x = tf.Variable([4., 2.], trainable=False) # gradient 진행을 안 합니다. 



In [19]:
# 아래와 같이 nested with priority도 가능합니다. 
x = tf.constant(5.0)
with tf.GradientTape() as g:
  g.watch(x)
  with tf.GradientTape() as gg:
    gg.watch(x)
    y = x * x
  dy_dx = gg.gradient(y, x)  # dy_dx = 2 * x
d2y_dx2 = g.gradient(dy_dx, x)  # d2y_dx2 = 2

In [21]:
example = 0 # 예시
flag = 12 
with tf.GradientTape() as tape:
    los= example
    if los > flag:
        tape.reset() # reset은 tape에 정보를 초기화해줍니다.

In [ ]:
x = tf.constant(4.0)
with tf.GradientTape() as tape:
  with tape.stop_recording(): # 일시적으로 tape에 record를 중지하는 방식입니다.
    y = x ** 2
dy_dx = tape.gradient(y, x)
print(dy_dx)